In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Сборка датасета

Скачаем необходимые датасеты

Тот который получается в результате eda


In [ ]:
doctors = pd.read_csv('https://github.com/yyaroslavskiy/cuddly-chainsaw/raw/refs/heads/develop/eda/doctors_eda_result.csv',encoding='utf-8')

In [ ]:
doctors

In [ ]:
del doctors['Unnamed: 0']

In [ ]:
doctors_reviews = pd.read_csv('https://github.com/yyaroslavskiy/cuddly-chainsaw/raw/refs/heads/develop/eda/merge/doctors_review.csv')

In [ ]:
del doctors_reviews['Unnamed: 0']

Из временной памяти коллаба, можно заменить

In [ ]:
unique_addresses_with_coords = pd.read_csv('https://github.com/yyaroslavskiy/cuddly-chainsaw/raw/refs/heads/develop/parsing/api_yandex_geocoder/api_addresses.csv')

In [ ]:
unique_addresses_with_coords

In [ ]:
mean_review_rate = pd.read_csv('https://github.com/yyaroslavskiy/cuddly-chainsaw/raw/refs/heads/develop/eda/review_stat.csv')

In [ ]:
mean_review_rate

Соберем датасет для анализа.

Из doctors.csv возьмем комбинацию имя доктора-клиника, добавим ссылку, рейтинг, цену с одного ресурса и если есть среднюю, опыт, метро, специальности, с кем работает (дети/взрослые),

из датасета с географией - координаты, район и административный округ,

из датасета с отзывами - среднюю оценку из отзывов

In [ ]:
address_records = []

#проходимся по адресам сбера для каждого врача чтобы собрать детальную инфу по каждому адресу
for i in [1, 2, 3]:
    sber_mask = doctors[f'clinic_{i}_address_sber'].notna()
    for idx, row in doctors[sber_mask].iterrows():
        adress_match = unique_addresses_with_coords[unique_addresses_with_coords['address'] == row[f'clinic_{i}_address_sber']]
        review_match = mean_review_rate[mean_review_rate['doctor_link']==row['link_sber']]
        address_records.append({
            'address': row[f'clinic_{i}_address_sber'],
            'source': 'sber',
            'doctor_name': row['name'],
            'doctor_link' : row['link_sber'],
            'rating': row['rating'],
            'price_source': row['price_sber'],
            'price_avg': row['price'],
            'experience': row['experience'],
            'metro' : row[f'clinic_{i}_metro_sber'],
            'speciality' : row['speciality'],
            'is_kids' : row['is_kids'],
            'is_adults' : row['is_adults'],
            'latitude': adress_match['latitude'].iloc[0] if not adress_match.empty else None,
            'longitude': adress_match['longitude'].iloc[0] if not adress_match.empty else None,
            'review_rating': review_match['rate'].iloc[0] if not review_match.empty else None,
            'district': adress_match['district'].iloc[0] if not adress_match.empty else None,
            'area': adress_match['area'].iloc[0] if not adress_match.empty else None
        })


#проходимся по адресам продокторов для каждого врача
for i in [1, 2, 3]:
    prod_mask = doctors[f'clinic_{i}_address_prod'].notna()
    for idx, row in doctors[prod_mask].iterrows():
        adress_match = unique_addresses_with_coords[unique_addresses_with_coords['address'] == row[f'clinic_{i}_address_prod']]
        review_match = mean_review_rate[mean_review_rate['doctor_link']==row['link_prod']]
        address_records.append({
            'address': row[f'clinic_{i}_address_prod'],
            'source': 'prod',
            'doctor_name': row['name'],
            'doctor_link' : row['link_prod'],
            'rating': row['rating'],
            'price_source': row['price_prod'],
            'price_avg': row['price'],
            'experience': row['experience'],
            'metro' : row[f'clinic_{i}_metro_prod'],
            'speciality' : row['speciality'],
            'is_kids' : row['is_kids'],
            'is_adults' : row['is_adults'],
            'latitude': adress_match['latitude'].iloc[0] if not adress_match.empty else None,
            'longitude': adress_match['longitude'].iloc[0] if not adress_match.empty else None,
            'review_rating': review_match['rate'].iloc[0]if not review_match.empty else None,
            'district': adress_match['district'].iloc[0] if not adress_match.empty else None,
            'area': adress_match['area'].iloc[0] if not adress_match.empty else None
        })

data_coords = pd.DataFrame(address_records)

In [ ]:
data_coords

In [ ]:
data_coords.info()

In [ ]:
data_coords.describe()

In [ ]:
data_coords = data_coords.dropna(subset= 'latitude')

In [ ]:
data_coords= data_coords.drop_duplicates()

## Карты

### Визуализация Москвы

In [ ]:
sns.set(style='darkgrid')

Выставим границы для координат, чтобы получить только московские адреса


In [ ]:
data_moskow = data_coords[(data_coords['latitude']<55.931762) & (data_coords['latitude']>55.545493) & (data_coords['longitude']>37.344287) & (data_coords['longitude']<37.879998)]

In [ ]:
data_moskow.to_csv('data_moskow.csv')

In [ ]:
data_moskow = pd.read_csv('/content/data_moskow.csv')

In [ ]:
plt.figure(figsize=(9, 9))

plt.plot(data_moskow['longitude'], data_moskow['latitude'], 'ro', label='Клиники', alpha=0.1, color='maroon', markersize=5)

plt.xlabel('Долгота', fontsize=16)
plt.ylabel('Широта', fontsize=16)
plt.title('Карта клиник Москвы', fontsize=16)
plt.legend(fontsize=14, frameon=True)

# скачать изображение с прозрачным фоном
# plt.savefig('card_base.png', transparent=True)
plt.show()

границы москвы с сопоставлением с реальной картой

In [ ]:
data_moskow[(data_moskow['latitude']<55.58)&(data_moskow['longitude']<37.4)|(data_moskow['latitude']>55.92)&(data_moskow['longitude']>37.86)]

## Карты по округам

### Визуализация округов

In [ ]:
data_moskow['area'].unique()

In [ ]:
data_moskow['district'].unique()

In [ ]:
data_moskow=data_moskow.replace('Ошибка',np.nan)

In [ ]:
plt.figure(figsize=(15, 9))
unique_areas = data_moskow['area'].dropna().nunique()

for i, area in enumerate(data_moskow['area'].dropna().unique()):
    area_data = data_moskow[data_moskow['area'] == area]
    plt.scatter(area_data['longitude'], area_data['latitude'],
                   color=plt.cm.tab10(i/unique_areas), label=area, alpha=0.7, s=30)

plt.xlabel('Долгота', fontsize=16)
plt.ylabel('Широта', fontsize=16)
plt.title('Административные округа', fontsize=16)

# Чтобы разместить легенду за правым краем графика
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()

plt.savefig('card.png', transparent=True, bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
data_moskow[(data_moskow['latitude']>55.92)|(data_moskow['latitude']<55.55)]

Заменим на более короткие названия округов

In [ ]:
data_moskow_okr = data_moskow.copy()
data_moskow_okr['area'] = data_moskow_okr['area'].replace({
    'Центральный административный округ': 'ЦАО',
    'Северный административный округ': 'САО',
    'Северо-Восточный административный округ': 'СВАО',
    'Восточный административный округ': 'ВАО',
    'Юго-Восточный административный округ': 'ЮВАО',
    'Южный административный округ': 'ЮАО',
    'Юго-Западный административный округ': 'ЮЗАО',
    'Западный административный округ': 'ЗАО',
    'Северо-Западный административный округ': 'СЗАО'
    })

Посмотрим на статистики по округам


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

area_counts = data_moskow_okr['area'].value_counts()
axes[0,0].bar(area_counts.index, area_counts.values)
axes[0,0].set_title('Количество врачей по округам')
axes[0,0].tick_params(axis='x')

area_rating = data_moskow_okr.groupby('area')['rating'].mean().sort_values(ascending=False)
axes[0,1].bar(area_rating.index, area_rating.values)
axes[0,1].set_title('Средний рейтинг по округам')
axes[0,1].tick_params(axis='x')

area_price = data_moskow_okr.groupby('area')['price_avg'].mean().sort_values(ascending=False)
axes[1,0].bar(area_price.index, area_price.values)
axes[1,0].set_title('Средняя цена по округам')
axes[1,0].tick_params(axis='x')

area_med_price = data_moskow_okr.groupby('area')['price_avg'].median().sort_values(ascending=False)
axes[1,1].bar(area_med_price.index, area_med_price.values)
axes[1,1].set_title('Медианная цена по округам')
axes[1,1].tick_params(axis='x')

plt.tight_layout()
plt.show()

Заметим, что ЗАО выделяется по цене и рейтингу, вероятно, в этом округе более престижная медицина

Также в ЦАО сконцентрировано наибольшее количество врачей, и является одним из самых высоких среди средних рейтингов и цен

ВАО, СВАО, ЮВАО, СЗАО - наименьшие цены и количество докторов - возможно меньший спрос

### Карта рейтингов по округам

С цветовыми точками, в зависимости от рейтинга

In [ ]:
area_avg_ratings = data_moskow.groupby('area')['rating'].mean()

plt.figure(figsize=(12, 9))
scatter = plt.scatter(data_moskow['longitude'], data_moskow['latitude'],
                     c=data_moskow['rating'], cmap='RdYlGn', s=30, alpha=0.7, linewidth=0.3,vmax = 6)

cbar = plt.colorbar(scatter)
cbar.set_label('Рейтинг врача', fontsize=12)

areas = {
    'Центральный': (37.621184, 55.753600),
    'Северный': (37.525774, 55.838390),
    'Северо-Восточный': (37.632565, 55.854875),
    'Восточный': (37.775631, 55.787715),
    'Юго-Восточный': (37.754592, 55.692019),
    'Южный': (37.678065, 55.622014),
    'Юго-Западный': (37.576187, 55.662735),
    'Западный': (37.443533, 55.728003),
    'Северо-Западный': (37.451555, 55.829370),
    'Новомосковский': (37.371452,55.594351),
    'Троицкий': (37.146999,55.355802),
    'Зеленоградский': (37.194250,55.987583)
}

for area, (lon, lat) in areas.items():
    full_area_name = area + ' административный округ'
    if full_area_name in area_avg_ratings.index:
        avg_rating = area_avg_ratings[full_area_name]
        plt.annotate(f"{area}\n{avg_rating:.2f}", (lon, lat), fontsize=9, ha='center', va='center',
                    bbox=dict(boxstyle="round,pad=0.4", facecolor='white', alpha=0.8))

plt.xlabel('Долгота', fontsize=12)
plt.ylabel('Широта', fontsize=12)
plt.title('Карта рейтингов врачей по административным округам Москвы', fontsize=14)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('card_2.png', transparent=True, bbox_inches='tight', dpi=300)
plt.show()

Самые низкие показатели имеет Восточный район, а высокие - Западный и Центральный

In [ ]:
data_moskow.groupby('area')['rating'].mean().sort_values()

In [ ]:
plt.figure(figsize=(9, 9))

# Группируем по координатам и считаем средний рейтинг
grouped_data = data_moskow.groupby(['longitude', 'latitude'])['rating'].mean().reset_index()

scatter = plt.scatter(grouped_data['longitude'], grouped_data['latitude'],
                     c=grouped_data['rating'], cmap='RdYlGn', alpha=1, s=30, vmax = 5)

plt.xlabel('Долгота', fontsize=16)
plt.ylabel('Широта', fontsize=16)
plt.title('Карта клиник по среднему рейтингу врачей', fontsize=16)
plt.colorbar(scatter, label='Средний рейтинг')

plt.show()

Не очень репрезентативно, конкретная оценка рейтинга не сильно зависит от района, присутствуют, как высокие так и низкие, сильной цветовой концентрации нет, можно выделить только, что на Юго-Западе меньше низких оценок

Построим тепловыую карту, объединив близлежащие области

In [ ]:
plt.figure(figsize=(12,10))

cut_data = data_moskow.copy()
# Округляем координаты для группировки в ячейки сетки
# Умножаем на 4, округляем, делим на 4 - получаем сетку с шагом 0.25
cut_data['latitude'] = np.round(cut_data['latitude']*4, decimals=1)*0.25
cut_data['longitude'] = np.round(cut_data['longitude']*4, decimals=1)*0.25

table = pd.pivot_table(cut_data, values='rating', index=['latitude'],columns=['longitude'], aggfunc=np.mean)
table = table.sort_index(ascending=False)
table_count = pd.pivot_table(cut_data, values='rating', index=['latitude'], columns=['longitude'], aggfunc=np.size)
table_count = table_count.sort_index(ascending=False)
sns.heatmap(table, mask = table_count<4, cmap='RdYlGn', annot=True)

plt.xlabel('Долгота', fontsize=16)
plt.ylabel('Широта', fontsize=16)
plt.title('Хитмап по рейтингу', fontsize=16)

plt.show()

При небольшом объединении заметно ухудшение оценок на границах мкада, а также на Востоке, Юго-Востоке

### Карта Рейтинг с отзывов

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Группируем по координатам и считаем средний рейтинг
grouped_data_1 = data_moskow.groupby(['longitude', 'latitude'])['review_rating'].mean().reset_index()

scatter0 = axes[0].scatter(grouped_data_1['longitude'], grouped_data_1['latitude'],
                     c=grouped_data_1['review_rating'], cmap='RdYlGn', alpha=1, s=30, vmax=6)
axes[0].set_xlabel('Долгота', fontsize=16)
axes[0].set_ylabel('Широта', fontsize=16)
axes[0].set_title('Средний рейтинг по отзывам', fontsize=16)
plt.colorbar(scatter0, ax=axes[0], label='Средний рейтинг по отзывам')

low_ratings = grouped_data_1[grouped_data_1['review_rating'] < 3]
scatter1 = axes[1].scatter(low_ratings['longitude'], low_ratings['latitude'],
                     c=low_ratings['review_rating'], cmap='YlOrRd_r', alpha=1, s=30, vmin=1, vmax=4)
axes[1].set_xlabel('Долгота', fontsize=16)
axes[1].set_ylabel('Широта', fontsize=16)
axes[1].set_title('Средний рейтинг по отзывам ниже 3', fontsize=16)
plt.colorbar(scatter1, ax=axes[1], label='Средний рейтинг по отзывам')

plt.tight_layout()
plt.savefig('card_3.png', transparent=True, bbox_inches='tight', dpi=300)
plt.show()

среднюю оценку из отзывов все округа имеют высокие показатели >= 4.39, однако заметно достаточное кол-во отрицательных оценок

Рейтинг также не сильно зависит от района, можно выделить только, что на Юге почти отсутствую низкие оценки

### Карта по цене

Рассмотрим цену

In [ ]:
Q1 = data_moskow['price_avg'].quantile(0.25)
Q3 = data_moskow['price_avg'].quantile(0.75)
IQR = Q3 - Q1
lower_whisker = Q1 - 1.5 * IQR
upper_whisker = Q3 + 1.5 * IQR
print(lower_whisker,upper_whisker)

In [ ]:
data_moskow['price_avg'].describe()

уберем значения > Q3+3IQR на данном графике

In [ ]:
plt.figure(figsize=(9, 9))

scatter = plt.scatter(data_moskow[data_moskow['price_avg'] < upper_whisker]['longitude'], data_moskow[data_moskow['price_avg'] < upper_whisker]['latitude'],
                     c=data_moskow[data_moskow['price_avg'] < upper_whisker]['price_avg'], cmap='RdYlGn_r',
                     alpha=0.7, s=30, linewidth=0.3)

plt.xlabel('Долгота', fontsize=16)
plt.ylabel('Широта', fontsize=16)
plt.title('Карта врачей по цене приема', fontsize=16)
plt.colorbar(scatter, label='Цена')
plt.show()

по графику замечаем, что на юге-востоке и периферии цены ниже, а в центре и северо-западе цены выше среднего и медианы

In [ ]:
plt.figure(figsize=(14, 10))

scatter = plt.scatter(data_moskow[data_moskow['price_avg'] < Q3+3*IQR]['longitude'], data_moskow[data_moskow['price_avg'] < Q3+3*IQR]['latitude'],
                     c=data_moskow[data_moskow['price_avg'] < Q3+3*IQR]['price_avg'], cmap='RdYlGn_r', s=30, alpha=0.7, linewidth=0.3)

cbar = plt.colorbar(scatter)
cbar.set_label('цена приема', fontsize=12)

areas = {
    'Центральный': (37.621184, 55.753600),
    'Северный': (37.525774, 55.838390),
    'Северо-Восточный': (37.632565, 55.854875),
    'Восточный': (37.775631, 55.787715),
    'Юго-Восточный': (37.754592, 55.692019),
    'Южный': (37.678065, 55.622014),
    'Юго-Западный': (37.576187, 55.662735),
    'Западный': (37.443533, 55.728003),
    'Северо-Западный': (37.451555, 55.829370),
    'Новомосковский': (37.371452,55.594351),
    'Троицкий': (37.146999,55.355802),
    'Зеленоградский': (37.194250,55.987583)
}

area_avg_prices = data_moskow[data_moskow['price_avg'] < Q3+3*IQR].groupby('area')['price_avg'].mean()
for area, (lon, lat) in areas.items():
    full_area_name = area + ' административный округ'
    if full_area_name in area_avg_prices.index:
        avg_price = area_avg_prices[full_area_name]
        plt.annotate(f"{area}\n{avg_price:.0f}р", (lon, lat), fontsize=9, ha='center',
                    bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.9))

plt.xlabel('Долгота', fontsize=12)
plt.ylabel('Широта', fontsize=12)
plt.title('Карта средних цен по административным округам Москвы', fontsize=16)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('card_4.png', transparent=True, bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
plt.figure(figsize=(12,10))

cut_data = data_moskow.copy()
# Округляем координаты для группировки в ячейки сетки
cut_data['latitude'] = np.round(cut_data['latitude']*4, decimals=1)*0.25
cut_data['longitude'] = np.round(cut_data['longitude']*4, decimals=1)*0.25

table = pd.pivot_table(cut_data, values='price_avg', index=['latitude'],columns=['longitude'], aggfunc=np.mean)
table = table.sort_index(ascending=False)
table_count = pd.pivot_table(cut_data, values='price_avg', index=['latitude'], columns=['longitude'], aggfunc=np.size)
table_count = table_count.sort_index(ascending=False)

ax = sns.heatmap(table, mask = table_count<4, cmap='RdYlGn_r', vmax = 10500)

areas = {
    'Центральный': (37.621184, 55.753600),
    'Северный': (37.525774, 55.838390),
    'Северо-Восточный': (37.632565, 55.854875),
    'Восточный': (37.775631, 55.787715),
    'Юго-Восточный': (37.754592, 55.692019),
    'Южный': (37.678065, 55.622014),
    'Юго-Западный': (37.576187, 55.662735),
    'Западный': (37.443533, 55.728003),
    'Северо-Западный': (37.451555, 55.829370),
    'Новомосковский': (37.371452,55.594351),
    'Троицкий': (37.146999,55.355802),
    'Зеленоградский': (37.194250,55.987583)
}

area_avg_prices = data_moskow[data_moskow['price_avg'] < Q3+3*IQR].groupby('area')['price_avg'].mean()
for area, (lon, lat) in areas.items():
    full_area_name = area + ' административный округ'
    if full_area_name in area_avg_prices.index:
        avg_price = area_avg_prices[full_area_name]

        # Преобразуем географические координаты в координаты ячеек хитмапа, тк там строится по индексам
        lat_idx = np.round(lat*4, decimals=1)*0.25
        lon_idx = np.round(lon*4, decimals=1)*0.25

        x_pos = list(table.columns).index(lon_idx)
        y_pos = list(table.index).index(lat_idx)

        ax.annotate(f"{area}\n{avg_price:.0f}р",
                    (x_pos, y_pos),
                    fontsize=10,
                    ha='center',
                    va='center',
                    bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.7))

plt.xlabel('Долгота', fontsize=16)
plt.ylabel('Широта', fontsize=16)
plt.title('Хитмап по цене', fontsize=16)

plt.tight_layout()
plt.show()

Стоит отметить, что самая высокая цена не в центральном районе, а в Западном округе, далее в Юго-Западном. Минимальная средняя цена в Северно-Западном.

In [ ]:
area_avg_prices = data_moskow[data_moskow['price_avg'] < Q3+3*IQR].groupby('area')[['price_avg', 'rating']].mean().sort_values(by = 'price_avg', ascending = False)

In [ ]:
area_avg_prices

посмотрим на выбросы по ценам

In [ ]:
plt.figure(figsize=(14, 10))

scatter = plt.scatter(data_moskow[data_moskow['price_avg'] >= Q3+3*IQR]['longitude'], data_moskow[data_moskow['price_avg'] >= Q3+3*IQR]['latitude'],
                     c=data_moskow[data_moskow['price_avg'] >= Q3+3*IQR]['price_avg'], cmap='viridis', s=30, alpha=0.7, linewidth=0.3)

cbar = plt.colorbar(scatter)
cbar.set_label('цена приема', fontsize=12)

areas = {
    'Центральный': (37.621184, 55.753600),
    'Северный': (37.525774, 55.838390),
    'Северо-Восточный': (37.632565, 55.854875),
    'Восточный': (37.775631, 55.787715),
    'Юго-Восточный': (37.754592, 55.692019),
    'Южный': (37.678065, 55.622014),
    'Юго-Западный': (37.576187, 55.662735),
    'Западный': (37.443533, 55.728003),
    'Северо-Западный': (37.451555, 55.829370),
    'Новомосковский': (37.371452,55.594351),
    'Троицкий': (37.146999,55.355802),
    'Зеленоградский': (37.194250,55.987583)
}

area_avg_prices = data_moskow[data_moskow['price_avg'] >= Q3+3*IQR].groupby('area')['price_avg'].mean()
for area, (lon, lat) in areas.items():
    full_area_name = area + ' административный округ'
    if full_area_name in area_avg_prices.index:
        avg_price = area_avg_prices[full_area_name]
        plt.annotate(f"{area}\n{avg_price:.0f}р", (lon, lat), fontsize=9, ha='center',
                    bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.5))

plt.xlabel('долгота', fontsize=12)
plt.ylabel('широта', fontsize=12)
plt.title('карта выбросов средних цен по административным округам Москвы', fontsize=16)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

Выбросы сконцентрированы в Центральном, Западном и Юго-Западном районе

In [ ]:
area_avg_prices = data_moskow[data_moskow['price_avg'] >= Q3+3*IQR].groupby('area')['price_avg'].mean()

In [ ]:
data_moskow['price_avg'].describe()

## Карта по пациентам

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

kids_clinics = data_moskow[data_moskow['is_kids'] == True]
adults_clinics = data_moskow[data_moskow['is_adults'] == True]
both_clinics = data_moskow[(data_moskow['is_kids'] == True) & (data_moskow['is_adults'] == True)]

axes[0].plot(adults_clinics['longitude'], adults_clinics['latitude'], 'o', alpha=0.6, color='blue', markersize=3)
axes[0].set_title('Врачи для взрослых', fontsize=14)
axes[0].set_xlabel('Долгота')
axes[0].set_ylabel('Широта')
axes[0].grid(alpha=0.4)

axes[1].plot(kids_clinics['longitude'], kids_clinics['latitude'], 'o', alpha=0.6, color='red', markersize=3)
axes[1].set_title('Врачи для детей', fontsize=14)
axes[1].set_xlabel('Долгота')
axes[1].set_ylabel('Широта')
axes[1].grid(alpha=0.4)

axes[2].plot(both_clinics['longitude'], both_clinics['latitude'], 'o', alpha=0.6, color='green', markersize=3)
axes[2].set_title('Врачи для детей и взрослых', fontsize=14)
axes[2].set_xlabel('Долгота')
axes[2].set_ylabel('Широта')
axes[2].grid(alpha=0.4)

plt.tight_layout()
plt.show()

Нельзя выделить закономерности или сильную концентрацию с определенных районах,  концентрация совпадает с общим количеством докторов

## Районы

Заменим все значения, содержащие "микрорайон", так слишком узкая выборка

In [ ]:
data_moskow['district'] = data_moskow['district'].replace({'Ошибка': None})
data_moskow.loc[data_moskow['district'].str.contains('микрорайон', case=False, na=False), 'district'] = None

Посмотрим по районам максимальные по признакам: количество врачей, рейтинг, цена

In [ ]:
bubble_data = data_moskow.groupby('district').agg({
    'doctor_name': 'count',
    'rating': 'mean',
    'price_avg': 'median'
}).reset_index()

bubble_data_doctor_name = bubble_data.sort_values('doctor_name', ascending = False)
bubble_data_doctor_name.head(15)

Максимальные по количеству врачей:

район Хамовники

Мещанский район

Тверской район

Таганский район

In [ ]:
bubble_data_rating = bubble_data.sort_values('rating', ascending = False)
bubble_data_rating.head(15)

Максимальные по рейтингу:

район Братеево

район Очаково-Матвеевско

район Восточное Дегунино

район Ивановское

In [ ]:
bubble_data_price_final = bubble_data.sort_values('price_avg', ascending = False)
bubble_data_price_final.head(15)

Максимальные по цене:

район Беговой

район Проспект Вернадского

Обручевский район

район Черёмушки

### Bubble diagram

Совместим три переменные и посмотрим на графике

In [ ]:
bubble_data_mean = data_moskow.groupby('district').agg({
    'doctor_name': 'count',
    'rating': 'mean',
    'price_avg': 'mean'
}).reset_index()

bubble_data_mean_doctor_name = bubble_data.sort_values('doctor_name', ascending = False)
bubble_data_mean_doctor_name.head(15)

In [ ]:
import plotly.express as px

top_10_names = bubble_data.nlargest(10, 'doctor_name')['district'].tolist()
bubble_data['legend_group'] = bubble_data['district'].apply(
    lambda x: x if x in top_10_names else 'Другие районы'
)

fig = px.scatter(bubble_data,
                 x='rating',
                 y='price_avg',
                 size='doctor_name',
                 size_max=50,
                 color='legend_group',
                 color_discrete_sequence=px.colors.qualitative.Dark24,
                 hover_name='district',
                 hover_data={
                     'rating': ':.2f',
                     'price_avg': ':.0f',
                     'doctor_name': True
                 },
                 title='Сравнение медианной цены и рейтинга по району<br><sub>с учетом количества врачей в районе</sub>',
                 labels={
                     'rating': 'Средний рейтинг',
                     'price_avg': 'Медианная цена',
                     'doctor_name': 'Количество врачей',
                     'legend_group': 'Районы'
                 })

fig.update_layout(
    width=1350,
    height=890
)

fig.show()

In [ ]:
import plotly.express as px

top_10_names = bubble_data_mean.nlargest(10, 'doctor_name')['district'].tolist()
bubble_data_mean['legend_group'] = bubble_data_mean['district'].apply(
    lambda x: x if x in top_10_names else 'Другие районы'
)

fig = px.scatter(bubble_data_mean,
                 x='rating',
                 y='price_avg',
                 size='doctor_name',
                 size_max=50,
                 color='legend_group',
                 color_discrete_sequence=px.colors.qualitative.Dark24,
                 hover_name='district',
                 hover_data={
                     'rating': ':.2f',
                     'price_avg': ':.0f',
                     'doctor_name': True
                 },
                 title='Сравнение средней цены и рейтинга по району<br><sub>с учетом количества врачей в районе</sub>',
                 labels={
                     'rating': 'Средний рейтинг',
                     'price_avg': 'Средняя цена',
                     'doctor_name': 'Количество врачей',
                     'legend_group': 'Районы'
                 })

fig.update_layout(
    width=1350,
    height=890
)

fig.show()

# Вывод

1. Плотность расположения

На карте Москвы наблюдается высокая концентрация клиник в ЦАО и Юго-Западных районах, а также вдоль основных транспортных магистралей. Районы с такими же высокими показателями относятся к этим округам.

2. Стоимость

Наиболее высокая стоимость наблюдается в ЦАО, ЮЗАО и ЗАО, там же больше всего выбросов по цене. Районы с такими же высокими показателями относятся к этим округам.

ВАО, СВАО, ЮВАО, СЗАО - наименьшие цены. Районы в целом аналогично.

3. Рейтинги

В среднем самые низкие показатели имеет Восточный округ, а высокие - Западный и Центральный. Однако по отдельным районам возможны противоположные результаты.

Рейтинг по отзывам в среднем имеет относительно высокую оценку >= 4.39, распределен равномерно районам, однако в ЮЗАО заметно меньшее количество плохих оценок.

4. Зависимости от опыта, специальности, типу пациентов не обнаружено